refer: 
@gru 구조
https://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html

@data loading
https://github.com/RobRomijnders/LSTM_tsc

### import packages

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="4" #4번 DEVICES만 사용하여 다른 메모리를 낭비를 방지
import time

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
% matplotlib inline
#ipython 내에 그래프가 보이게함

from sklearn.model_selection import train_test_split

In [2]:
tf.reset_default_graph()
saver = tf.train.import_meta_graph("./180122_3.ckpt.meta")

learning_rate = 0.001
total_epoch = 1000
batch_size = 100 
dropout = 0.5
hidden_size = 200

In [3]:
class Timenet:
    
    def __init__(self, sess, dataset):
        print("===============================================================  ", dataset)
        self.sess = sess
        self.dataset = dataset
        self.model(self.dataset)
        
    def model(self, dataset):
        datadir = '/home/sohee/UCR_TS_Archive_2015' + '/' + dataset + '/' + dataset
        data_train = np.loadtxt(datadir+'_TRAIN', delimiter=',')
        data_test = np.loadtxt(datadir+'_TEST', delimiter=',')

        #train data
        self.X_train = data_train[:,1:]
        self.y_train = data_train[:,0]
        print("train data's size :",self.X_train.shape[0])
        y_train_first_index = int(np.unique(self.y_train)[0])
        if int(y_train_first_index) == 1:
            self.y_train = data_train[:,0]-1

        #test data
        self.X_test = data_test[:,1:]
        self.y_test = data_test[:,0]
        print("test data's size :",self.y_test.shape[0])
        y_test_first_index = int(np.unique(self.y_test)[0])
        if int(y_test_first_index) == 1:
            self.y_test = data_test[:,0]-1

        n_variable = self.X_train.shape[1]
        
        # placeholder
        self.encoder_inputs = tf.placeholder(tf.float32, [None, n_variable], name="encoder_inputs") 
        self.decoder_inputs = tf.placeholder(tf.float32, [None, n_variable], name="decoder_inputs")
        self.targets = tf.placeholder(tf.int64, [None], name="targets")

        with tf.variable_scope("seq2seq"+dataset):  
            cell = tf.contrib.rnn.GRUCell(num_units=hidden_size)
            outputs, states = tf.contrib.legacy_seq2seq.basic_rnn_seq2seq([self.encoder_inputs], [self.decoder_inputs], cell)
            outputs = tf.reshape(outputs, [-1, hidden_size]) #3D -> 2D # output 모양이 항상 [? , hidden_size]으로 고정됨

            #W = tf.Variable(tf.random_normal([hidden_size, 100]), name="W")
            #b = tf.Variable(tf.random_normal([100]), name="b")
            W = tf.get_default_graph().get_tensor_by_name("W:0")
            b = tf.get_default_graph().get_tensor_by_name("b:0")
            logits = tf.matmul(outputs, W) + b

        with tf.variable_scope("cost"+dataset):
            self.cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=self.targets))
            self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.cost) 

        with tf.variable_scope("eval"+dataset):
            prediction = tf.argmax(tf.nn.softmax(logits), 1) 
            correct_prediction = tf.equal(prediction, self.targets)#one-hot을 안쓰면 target에는 argmax할 필요없음
            self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
    def try_optimize(self):
        return self.sess.run([self.cost, self.optimazer2], feed_dict={ self.encoder_inputs : self.X_train, 
                                                                      self.decoder_inputs : self.X_train, 
                                                                      self.targets : self.y_train })
    
    def acc_calculator(self):
        return self.sess.run( self.accuracy, feed_dict={ self.encoder_inputs : self.X_test, 
                                                        self.decoder_inputs : self.X_test, 
                                                        self.targets : self.y_test })

In [4]:
testdatasets = ['synthetic_control','PhalangesOutlinesCorrect', 'DistalPhalanxOutlineAgeGroup', 'DistalPhalanxOutlineCorrect', 'DistalPhalanxTW', 
                'MiddlePhalanxOutlineAgeGroup', 'MiddlePhalanxOutlineCorrect', 'MiddlePhalanxTW', 'ProximalPhalanxOutlineAgeGroup', 
                'ProximalPhalanxOutlineCorrect', 'ProximalPhalanxTW', 'ElectricDevices', 'MedicalImages', 'SwedishLeaf', 'Two_Patterns', 'ECG5000', 
                'ECGFiveDays', 'ChlorineConcentration', 'Adiac', 'Strawberry', 'Cricket_X', 'Cricket_Y', 'Cricket_Z', 'uWaveGestureLibrary_X', 
                'uWaveGestureLibrary_Y', 'uWaveGestureLibrary_Z', 'yoga']

for dataset in testdatasets :   
    sess = tf.Session()
    m = Timenet(sess, dataset)
    
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, "./180122_3.ckpt")
    print("... check point loaded")

    start_time = time.time()
    
    for epoch in range(total_epoch):

        cost, _= sess.run( [m.cost, m.optimizer], feed_dict={ m.encoder_inputs : m.X_train, m.decoder_inputs : m.X_train, m.targets : m.y_train })
        
        if epoch == 0:  
            print("Epoch {}/1000 ".format(epoch+1), "cost : ", cost)
        if epoch == (total_epoch//2-1) :  
            print("Epoch {}/1000 ".format(epoch+1), "cost : ", cost)
        if epoch == (total_epoch-1) :  
            print("Epoch {}/1000 ".format(epoch+1), "cost : ", cost) 
            
    print(" It took ", time.time()-start_time, "s.")
    
    acc = m.acc_calculator()
    print("")
    print(dataset, ', Accuracy:', acc, ', Error rate : ', 1-acc, " with Transfer Learning ")
    print("")

===============================================================   synthetic_control
train data's size : 300
test data's size : 300
INFO:tensorflow:Restoring parameters from ./180122_3.ckpt
... check point loaded
Epoch 1/1000  cost :  5.77494
Epoch 500/1000  cost :  0.00026516
Epoch 1000/1000  cost :  8.32939e-05
 It took  31.070587158203125 s.

synthetic_control , Accuracy: 0.85 , Error rate :  0.149999976158  with Transfer Learning 

===============================================================   PhalangesOutlinesCorrect
train data's size : 1800
test data's size : 858
INFO:tensorflow:Restoring parameters from ./180122_3.ckpt
... check point loaded
Epoch 1/1000  cost :  8.9481
Epoch 500/1000  cost :  0.479623
Epoch 1000/1000  cost :  0.345472
 It took  141.74941492080688 s.

PhalangesOutlinesCorrect , Accuracy: 0.786713 , Error rate :  0.213286697865  with Transfer Learning 

===============================================================   DistalPhalanxOutlineAgeGroup
train data's s